In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import os

29-08-2025

In this jupyter notebook the metadata from the Airtel-Tigo network in Ghana is coupled to the signal level data from that network. 

This notebook also serves as a manual/README to reproduce these steps. 

The output of this notebook is a text file in the format that can be readily processed by RAINLINK. 

Note: RAINLINK can not (yet) handle data in OpenSense format (https://github.com/OpenSenseAction/OS_data_format_conventions) as input. 


## Overview of the data files
We received two Excel files with metadata from AT. <br>
1. *Microwave_Link_Report_*: contains almost all of the required metadata, except the coordinates of the sites. 
2. *Consolidated_Data*: contains all the required metadata, including coordinates, but has no way to distinguish between the horizontal and vertical polarizations in signal level data. 

The signal level data (received signal level - RSL, and transmitted signal level - TSL) are found on the ftp server from AT.  


### Loading the metadata

We primarily use the metadata from *Microwave_Link_Report_* because the format of the ID's matches those in the signal level data file the best. We only use the *Consolidated_Data* to match the coordinates with the rest of the metadata. 

**Note:** the file *Consolidated_Data* contains much more metadata than is required as input for RAINLINK, like ```Elevation```, ```Antenna Diameter```, ```Antenna Azimuth```, or ```Vertical Angle```. At this moment we do not use any of this data, but it could potentially be useful for additional analyses down the road. 

**Also note:** from row 308 onwards the columns in the original *Consolidated_Data* file were shifted to the right by one column, so we modified the file by shifting the columns back to align all data in the correct columns. 

In [2]:
metadata_file_path = '../data/280425/Microwave_Link_Report_27-05-2025_20-49-04.xlsx'
metadata_raw = pd.read_excel(metadata_file_path, skiprows=6, header=1)

coordinates_file_path = '../data/280425/consolidated_data_modified.xlsx'
coordinates_raw = pd.read_excel(coordinates_file_path, header=1)

metadata_vars = ['Source NE Name', 
                 'Sink NE Name',
                 'Source XPIC Polarization Direction',
                 'Sink XPIC Polarization Direction',
                 'Source NE Frequency (MHz)',
                 'Sink NE Frequency (MHz)',
                 'Source ODU Board/Port/Path',
                 'Sink ODU Board/Port/Path',]

coordinates_vars = ['Link ID', 
                    'Link Name',
                    'Site 1 ID',
                    'Site 2 ID', 
                    'Site 1 Latitude',
                    'Site 2 Latitude', 
                    'Site 1 Longitude', 
                    'Site 2 Longitude',
                    'Distance(km)',
                    'ATPC(dB)', # not used explicitly in RAINLINK but important to know which links ATCP is applied on
                    'Site 1 TX Freq(MHz)', # to double check the frequencies are the same in both metadata files
                    'Site 2 TX Freq(MHz)' # to double check the frequencies are the same in both metadata files
                    ]


metadata = metadata_raw[metadata_vars]
coordinates = coordinates_raw[coordinates_vars]

## Combining the metadata files
The metadata files from the mobile network operators are typically generated from a site (node) point of view, whereas for rainfall estimation we are typically interested in what happens along a link. Since most often a site is both a transmitting and receiving node we consider a link between two sites to consists of at least two sub-links, one from Site 1 to Site 2, and another from Site 2 to Site 1. 
All links in the current network appear to be dual-polarized (see columns ```Site 1 Antenna Polarization``` and ```Site 2 Antenna Polarization``` in *Consolidated_Data*) so per sub-link we have 2 channels, a vertically and a horizontally polarized one. 
In RAINLINK each polarization (channel) within a sub-link is treated as a separate rainfall sensor. We therefore need to create a metadata table with a separate entry for each polarization within a sub-link. 


*Frequency & polarization* <br>
Since RAINLINK requires a unique ID for each and every connection between two sites, when coupling the metadata to the signal level data we use a concatenation of ```Source NE Name``` and ```Source ODU Board/Port/Path``` from *Microwave_Link_Report_*, which is unique for each polarization within a sub-link.

**Note**: This assumption is based on the smaller set of RSL and TSL data previously shared in the *TAHMO_Data* Excel file. In this file the so-called ```Monitored Object``` is unique, and built up as follows:
(e.g. ID ```KUMOFF_RTN950A-1-23-MODU-1(RTNRF-1)-1```): 
- ```KUMOFF_RTN950A``` refers to a general location / tower, as multiple IDs can carry this part
- ```-1-``` is used to differentiate between radio units or antennas, if there are more than 1 antenna on that site. We assume this because different numbers (-1-, -2-, etc.) can have different frequencies, whereas the same numbers have the same frequencies. 
- ```23``` refers to a board within a radio unit, possibly using the same antenna. Often another number (24 or 25) has the same specs but is in so-called "stand-by" mode.  
- ```MODU-1(RTNRF-1)``` refers to the polarization (1=V, 2=H). 
- The last ```1``` in the ID is the same in all ID's so doesn't appear to have much of a function.

The ftp data files with RSL and TSL data contain the columns ```NEName```-```BrdID```-```BrdName```-```PortNO(PortName)```-```PathID``` whose entries can be matched to pattern of the two concatenated columns ```Source NE Name``` and ```Source ODU Board/Port/Path``` from *Microwave_Link_Report_*. This similarity in naming between *Microwave_Link_Report_* and txt files on the ftp server allows the metadata (frequency and polarization) to be matched with the corresponding RSL and TSL data. 

*Coordinates* <br>
The naming convention in *Consolidated_Data* is slightly different. To match the frequency and polarization with the correct site coordinates we use the first part (all caps, no special characters or spaces included) of the strings in columns ```Source NE Name``` and ```Sink NE Name``` from *Microwave_Link_Report_* to match the columns ```Site 1 ID``` and ```Site 2 ID``` in *Consolidated_Data*. As a double check, we also make sure that the frequencies between the two metadata files match. 


In [90]:
display(metadata)

,Source NE Name,Sink NE Name,Source XPIC Polarization Direction,Sink XPIC Polarization Direction,Source NE Frequency (MHz),Sink NE Frequency (MHz),Source ODU Board/Port/Path,Sink ODU Board/Port/Path
0,AYETEP_RTN950A,SEGE_RTN950A,V,V,8103.495,7792.175,22-MODU-1(RTNRF-1)-1,21-MODU-1(RTNRF-1)-1
1,AKMDAN BAR_RTN950A,DERMA BAR_RTN950A,V,V,8103.495,7792.175,25-MODU-1(RTNRF-1)-1,23-MODU-1(RTNRF-1)-1
2,WEIJA_RTN950A,SENYA_RTN950A,V,V,6270.54,6018.5,23-MODU-1(RTNRF-1)-1,23-MODU-1(RTNRF-1)-1
3,WEIJA_RTN950A,SENYA_RTN950A,H,H,6270.54,6018.5,23-MODU-2(RTNRF-2)-1,23-MODU-2(RTNRF-2)-1
4,ANTO_RTN910A,SHAMA_RTN910A,V,V,5989.675,6241.715,27-MODU-1(RTNRF-1)-1,27-MODU-1(RTNRF-1)-1
...,...,...,...,...,...,...,...,...
884,AMASAM_RTN910A-2,ADEISO_RTN910A,V,V,7910.775,8222.095,27-MODU-1(RTNRF-1)-1,27-MODU-1(RTNRF-1)-1
885,AMASAM_RTN910A-2,ADEISO_RTN910A,H,H,7910.775,8222.095,27-MODU-2(RTNRF-2)-1,27-MODU-2(RTNRF-2)-1
886,KPASSA_RTN950A_2,DAMBAI_RTN950A,-,-,7910.775,5989.675,23-MODU-2(RTNRF-2)-1,23-MODU-2(RTNRF-2)-1
887,KPASSA_RTN950A_2,DAMBAI_RTN950A,-,-,7910.775,5989.675,25-MODU-1(RTNRF-1)-1,25-MODU-1(RTNRF-1)-1


In [91]:
display(coordinates)

,Link ID,Link Name,Site 1 ID,Site 2 ID,Site 1 Latitude,Site 2 Latitude,Site 1 Longitude,Site 2 Longitude,Distance(km),ATPC(dB),Site 1 TX Freq(MHz),Site 2 TX Freq(MHz)
0,AHIEST_KUMOFF,AHIEST_KUMOFF,AHIEST,KUMOFF,6.66450,6.673921,-1.591239,-1.619161,3.258,0.0,"18930_V,18930_H","17920_V,17920_H"
1,NSSUTA_TAKGBC,NaN,TAKGBC,NSSUTA,5.30212,5.303750,-2.008610,-1.993810,1.650,0.0,"18930_V,18930_H","17920_V,17920_H"
2,AWASO_SBKWAI,NaN,SBKWAI,AWASO,6.19661,6.246830,-2.320280,-2.287620,6.626,0.0,"6241.715_V,6241.715_H","5989.675_V,5989.675_H"
3,EFKUMA_GH0064,NaN,GH0064,EFKUMA,4.95896,4.928060,-1.727990,-1.754780,4.528,0.0,"17920_V,17920_H","18930_V,18930_H"
4,KNONGO_JUASO,KNONGO_JUASO,KNONGO,JUASO,6.65235,6.596528,-1.193620,-1.123686,9.895,0.0,"8103.495_V,8103.495_H","7792.175_V,7792.175_H"
...,...,...,...,...,...,...,...,...,...,...,...,...
366,WULENS_CHAMBA,WULENS_CHAMBA,WULENS,CHAMBA,8.65350,8.692610,0.001189,-0.140640,16.197,0.0,"7910.775_V,7910.775_H","8222.095_V,8222.095_H"
367,WOROBN_MPRAES,WOROBN_MPRAES,WROBN,MPRAES,6.48486,6.577220,-0.493310,-0.761380,31.359,0.0,"8103.495_V,8103.495_H","7792.175_V,7792.175_H"
368,YUNYOO_TEMAA,YUNYOO_TEMAA,YUNYOO,TEMAA,10.48230,10.471640,-0.016920,-0.133730,12.842,0.0,"6241.715_V,6241.715_H","5989.675_V,5989.675_H"
369,ZAARE_TANZUI,ZAARE_TANZUI,ZAARE,TANZUI,10.80888,10.791180,-0.864720,-0.857290,2.120,0.0,"18930_V,18930_H","17920_V,17920_H"


### Inconsistencies in metadata
There are a number of inconsistencies in the naming convention of the links in *Consolidated_Data*. 

1. In principle ```Site 1 ID``` and ```Site 2 ID``` are all capital letters, but sometimes the strings in those columns contain other characters as well, which don't match with the naming convention (all caps) in *Microwave_Link_Report_*. <br>
**Fix:** only take the first part of the string (no special characters or spaces) and capitalize all in ```Site 1 ID``` and ```Site 2 ID```. 


In [ ]:
# example of BOAKO which is BOAKO_gh0317 in Consolidated_Data
display(coordinates[coordinates['Site 1 ID'].str.contains('BOAKO')])
display(metadata[metadata['Sink NE Name'].str.contains('BOAKO')])

,Link ID,Link Name,Site 1 ID,Site 2 ID,Site 1 Latitude,Site 2 Latitude,Site 1 Longitude,Site 2 Longitude,Distance(km),ATPC(dB),Site 1 TX Freq(MHz),Site 2 TX Freq(MHz)
10,BOAKO_gh0317_SANKOR,BOAKO_gh0317_SANKOR,BOAKO_gh0317,SANKOR,6.389269,6.50442,-2.564611,-2.50227,14.482,0.0,"8103.495_V,8103.495_H","7792.175_V,7792.175_H"
68,BOAKO_gh0317_SWINSO,BOAKO_gh0317_SWINSO,BOAKO_gh0317,SWINSO,6.389269,6.42639,-2.564611,-2.70612,16.184,0.0,"8222.095_V,8222.095_H","7910.775_V,7910.775_H"


,Source NE Name,Sink NE Name,Source XPIC Polarization Direction,Sink XPIC Polarization Direction,Source NE Frequency (MHz),Sink NE Frequency (MHz),Source ODU Board/Port/Path,Sink ODU Board/Port/Path
446,WIAWSO_RTN950A,BOAKO_RTN950A,H,H,5989.675,6241.715,26-MODU-2(RTNRF-2)-1,26-MODU-2(RTNRF-2)-1
470,SANKOR_RTN950A,BOAKO_RTN950A,V,V,7792.175,8103.495,24-MODU-1(RTNRF-1)-1,24-MODU-1(RTNRF-1)-1
471,SANKOR_RTN950A,BOAKO_RTN950A,H,H,7792.175,8103.495,24-MODU-2(RTNRF-2)-1,24-MODU-2(RTNRF-2)-1
636,SWINSO_RTN950A,BOAKO_RTN950A,V,V,7910.775,8222.095,24-MODU-1(RTNRF-1)-1,23-MODU-1(RTNRF-1)-1
646,SWINSO_RTN950A,BOAKO_RTN950A,H,H,7910.775,8222.095,24-MODU-2(RTNRF-2)-1,23-MODU-2(RTNRF-2)-1


2. When matching the frequency and polarization to the coordinates some rows cannot be matched because the frequency between the two metadata files, used as a double check, does not match. After having performed an ```outer_join``` it appears that for some sub-links the frequencies from Site 1 and Site 2, in *Consolidated_Data* are swapped. In principle this is not a problem for the rainfall retrieval because the path-averaged rainfall rate is attributed to the center of the path, irregardless of the direction of the signal. However, to be consistent in the joining of the two metadata files, and retain as many links as possible we fix this. <br>
**Fix:** After joining frequency and polarization with coordinates, find the rows that didn't join because of unmatched frequencies. If for these rows the frequency from Site 2 to Site 1 matches the frequency from Site 1 to Site 2 exactly, swap these around. <br>
Note: it is not the columns ```Site 1 ID``` and ```Site 2 ID``` in *Consolidated_Data* that are swapped, as these are consistent with the coordinates throughout the entire file.

In [ ]:
# example of frequency swap between Site 1 PRMPRM and Site 2 AYETEP in the coordinates file
display(coordinates[coordinates['Site 2 ID'].str.contains('AYETEP')])
display(metadata[metadata['Source NE Name'].str.contains('AYETEP')])


,Link ID,Link Name,Site 1 ID,Site 2 ID,Site 1 Latitude,Site 2 Latitude,Site 1 Longitude,Site 2 Longitude,Distance(km),ATPC(dB),Site 1 TX Freq(MHz),Site 2 TX Freq(MHz)
60,AYETEP_PRMPRM,NaN,PRMPRM,AYETEP,5.72098,5.77992,0.10777,0.27288,19.415,0.0,"8222.095_V,8222.095_H","7910.775_V,7910.775_H"


,Source NE Name,Sink NE Name,Source XPIC Polarization Direction,Sink XPIC Polarization Direction,Source NE Frequency (MHz),Sink NE Frequency (MHz),Source ODU Board/Port/Path,Sink ODU Board/Port/Path
0,AYETEP_RTN950A,SEGE_RTN950A,V,V,8103.495,7792.175,22-MODU-1(RTNRF-1)-1,21-MODU-1(RTNRF-1)-1
6,AYETEP_RTN950A,PRMPRM_RTN950A,V,V,8222.095,7910.775,21-MODU-1(RTNRF-1)-1,22-MODU-1(RTNRF-1)-1
16,AYETEP_RTN950A,PRMPRM_RTN950A,H,H,8222.095,7910.775,21-MODU-2(RTNRF-2)-1,22-MODU-2(RTNRF-2)-1
27,AYETEP_RTN950A,SEGE_RTN950A,H,H,8103.495,7792.175,22-MODU-2(RTNRF-2)-1,21-MODU-2(RTNRF-2)-1


3. When matching the frequency and polarization to the coordinates some rows cannot be matched because the frequency between the two metadata files, used as a double check, does not match. After having performed an ```outer_join``` it appears that for some sub-links the frequencies in the two metadata files are almost the same, but not exactly. <br>
**Fix:** After joining frequency and polarization with coordinates, find the rows that didn't join because of unmatched frequencies. We apply a 'frequency buffer' where if the difference between the two different frequencies is less than 0.5GHz (500MHz), we match the coordinates to the polarization and frequency using the frequency from *Microwave_Link_Report_*. <br>
Note: The frequency is only used to determine the coefficient *a* and exponent *b* in the $k-R$ relation. However, different *a*, *b* parameters can have a significant effect on the path-averaged rain rate, especially at lower frequencies. Therefore we limit ourselves to a buffer of 0.5GHz (see Fig. 2 in https://doi.org/10.5194/amt-9-2425-2016 for reference). 

In [ ]:
# example of slight frequency mismatch between Site 1 TX Freq in coordinates file and Sink NE Frequency in metadata file
display(coordinates[coordinates['Site 1 ID'].str.contains('LAGBES')])
display(metadata[metadata['Sink NE Name'].str.contains('LAGBES')])

,Link ID,Link Name,Site 1 ID,Site 2 ID,Site 1 Latitude,Site 2 Latitude,Site 1 Longitude,Site 2 Longitude,Distance(km),ATPC(dB),Site 1 TX Freq(MHz),Site 2 TX Freq(MHz)
322,LAGBES_WAWALE,LAGBES_WAWALE,LAGBES,WAWALE,10.40354,10.337356,-0.56378,-0.799333,26.815,0.0,"8222.095_V,8222.095_H","7910.775_V,7910.775_H"


,Source NE Name,Sink NE Name,Source XPIC Polarization Direction,Sink XPIC Polarization Direction,Source NE Frequency (MHz),Sink NE Frequency (MHz),Source ODU Board/Port/Path,Sink ODU Board/Port/Path
455,GAMBAG_RTN910A,LAGBES_RTN910A,V,V,7792.175,8103.495,27-MODU-1(RTNRF-1)-1,27-MODU-1(RTNRF-1)-1
459,GAMBAG_RTN910A,LAGBES_RTN910A,H,H,7792.175,8103.495,27-MODU-2(RTNRF-2)-1,27-MODU-2(RTNRF-2)-1
853,WAWALE_RTN950A_2,LAGBES_RTN950A,V,V,7910.775,8222.094,24-MODU-1(RTNRF-1)-1,24-MODU-1(RTNRF-1)-1
856,WAWALE_RTN950A_2,LAGBES_RTN950A,H,H,7910.775,8222.094,24-MODU-2(RTNRF-2)-1,24-MODU-2(RTNRF-2)-1
857,WAWALE_RTN950A_2,LAGBES_RTN950A,V,V,7910.775,8222.094,26-MODU-1(RTNRF-1)-1,26-MODU-1(RTNRF-1)-1
861,WAWALE_RTN950A_2,LAGBES_RTN950A,H,H,7910.775,8222.094,26-MODU-2(RTNRF-2)-1,26-MODU-2(RTNRF-2)-1


4. Occasionally a Site ID is used in *Consolidated_Data* which does not follow the naming convention in *Microwave_Link_Report_* or is misspelled. When inspecting this manually we see that some other column in *Consolidated_Data* does match, so we can deduce that the coordinates belong to the correct sub-link. <br>
**Fix:** none (for now). Finding these cases manually is labour intensive, and there is no general condition that satisfies these outlier cases. In the current data set this concerns only 2 links anyway. 

In [ ]:
# example of mismatch in IDs between Site 1 ID (Wrobn Frst) in coordinates file and SINK NE Name (WOROBN) in metadata file
display(coordinates_raw[coordinates_raw['Site 1 ID'].str.contains('Wrobn')])
display(metadata[metadata['Sink NE Name'].str.contains('WOROBN')])

,Link ID,Link Name,Logic Link Name,Site 1 ID,Site 2 ID,Site 1 Name,Site 2 Name,Site 1 Latitude,Site 2 Latitude,Site 1 Longitude,...,Stage,Remarks,Region,Last Update By,Last Update Date,Link Implementation Status,Restoration Resource,Restoration RSL Check Result,Physical Link ID,Lock
367,WOROBN_MPRAES,WOROBN_MPRAES,NaN,Wrobn Frst,MPRAES,601885,Mpraeso_2,6.48486,6.57722,-0.49331,...,Unknown,NaN,Unknown,z00332775,2023-10-18 22:56,NaN,NaN,NaN,NaN,Unlocked


,Source NE Name,Sink NE Name,Source XPIC Polarization Direction,Sink XPIC Polarization Direction,Source NE Frequency (MHz),Sink NE Frequency (MHz),Source ODU Board/Port/Path,Sink ODU Board/Port/Path
798,MPRAES_RTN950A,WOROBN_RTN910A_1,-,-,7792.175,8103.495,22-MODU-1(RTNRF-1)-1,27-MODU-1(RTNRF-1)-1
848,MPRAES_RTN950A,WOROBN_RTN910A_1,-,-,7792.175,8103.495,22-MODU-2(RTNRF-2)-1,27-MODU-2(RTNRF-2)-1


### Join frequency, polarization, and coordinates 
To join the frequency and polarization of a certain sub-link to the correct coordinates of that sub-link we create a temporary ID ```Temp_ID``` made by joining the entries from ```Site 1 ID``` and ```Site 2 ID``` from *Consolidated_Data*. In this way each sub-link gets a unique set of coordinates. Naturally the different polarizations within a sub-link will have the same coordinates. <br>
As mentioned [previously](#combining-the-metadata-files) RAINLINK requires a unique ID for each and every connection between two sites. So when coupling the metadata to the TSL and RSL we will actually use a concatenation of ```Source NE Name``` and ```Source ODU Board/Port/Path``` from *Microwave_Link_Report_*, which is unique for each polarization. 


In [3]:
# Fix inconsistency 1: 
# Capitalize Site IDs and take only the first part of the string
coordinates.loc[:, 'Site 1 ID'] = coordinates['Site 1 ID'].str.extract(r'^([A-Za-z0-9]+)')[0].str.upper() 
coordinates.loc[:, 'Site 2 ID'] = coordinates['Site 2 ID'].str.extract(r'^([A-Za-z0-9]+)')[0].str.upper()

In [4]:
sublink1 = pd.DataFrame()
sublink1['Temp_ID'] = metadata['Source NE Name'].str.extract(r'^([A-Z0-9]+)') + '_' + metadata['Sink NE Name'].str.extract(r'^([A-Z0-9]+)') # dummy ID to match with coordinates
sublink1['Monitored_ID'] = metadata['Source NE Name'] + '-' + metadata['Source ODU Board/Port/Path']
sublink1['Far_end_ID'] = metadata['Sink NE Name'] + '-' + metadata['Sink ODU Board/Port/Path']
sublink1['Frequency'] = metadata['Source NE Frequency (MHz)']
sublink1['Polarization'] = metadata['Source XPIC Polarization Direction']

sublink2 = pd.DataFrame()
sublink2['Temp_ID'] = metadata['Sink NE Name'].str.extract(r'^([A-Z0-9]+)') + '_' + metadata['Source NE Name'].str.extract(r'^([A-Z0-9]+)')
sublink2['Monitored_ID'] = metadata['Sink NE Name'] + '-' + metadata['Sink ODU Board/Port/Path']
sublink2['Far_end_ID'] = metadata['Source NE Name'] + '-' + metadata['Source ODU Board/Port/Path']
sublink2['Frequency'] = metadata['Sink NE Frequency (MHz)']
sublink2['Polarization'] = metadata['Sink XPIC Polarization Direction']

sublink1_coordinates = pd.DataFrame()
sublink1_coordinates['Temp_ID'] = coordinates['Site 1 ID'] + '_' + coordinates['Site 2 ID']
sublink1_coordinates['XStart'] = coordinates['Site 1 Longitude']
sublink1_coordinates['YStart'] = coordinates['Site 1 Latitude']
sublink1_coordinates['XEnd'] = coordinates['Site 2 Longitude']
sublink1_coordinates['YEnd'] = coordinates['Site 2 Latitude']
sublink1_coordinates['PathLength'] = coordinates['Distance(km)']
sublink1_coordinates['ATPC'] = coordinates['ATPC(dB)']
sublink1_coordinates['Frequency'] = coordinates['Site 1 TX Freq(MHz)'].apply(lambda x: x.split(',')[0].split('_')[0]) # to double check the frequencies are the same in both metadata files

sublink2_coordinates = pd.DataFrame()
sublink2_coordinates['Temp_ID'] = coordinates['Site 2 ID'] + '_' + coordinates['Site 1 ID']
sublink2_coordinates['XStart'] = coordinates['Site 2 Longitude']
sublink2_coordinates['YStart'] = coordinates['Site 2 Latitude']
sublink2_coordinates['XEnd'] = coordinates['Site 1 Longitude']
sublink2_coordinates['YEnd'] = coordinates['Site 1 Latitude']
sublink2_coordinates['PathLength'] = coordinates['Distance(km)']
sublink2_coordinates['ATPC'] = coordinates['ATPC(dB)']
sublink2_coordinates['Frequency'] = coordinates['Site 2 TX Freq(MHz)'].apply(lambda x: x.split(',')[0].split('_')[0]) # to double check the frequencies are the same in both metadata files

# join the dataframes for the two sublinks
all_sublinks = pd.concat([sublink1, sublink2], ignore_index=True)
all_sublink_coordinates = pd.concat([sublink1_coordinates, sublink2_coordinates], ignore_index=True)

# enforce data types to be the same before merging
all_sublinks['Temp_ID'] = all_sublinks['Temp_ID'].astype(str)
all_sublinks['Frequency'] = pd.to_numeric(all_sublinks['Frequency'], errors='coerce')
all_sublink_coordinates['Temp_ID'] = all_sublink_coordinates['Temp_ID'].astype(str)
all_sublink_coordinates['Frequency'] = pd.to_numeric(all_sublink_coordinates['Frequency'], errors='coerce')

all_metadata = pd.merge(all_sublinks, all_sublink_coordinates, on=['Temp_ID'], how='outer', indicator=True)


In [5]:
# check how many rows match and mismatch in frequency
frequency_match = all_metadata[all_metadata['Frequency_x'] == all_metadata['Frequency_y']]
frequency_mismatch = all_metadata[all_metadata['Frequency_x'] != all_metadata['Frequency_y']]

display(frequency_match)
display(frequency_mismatch)

,Temp_ID,Monitored_ID,Far_end_ID,Frequency_x,Polarization,XStart,YStart,XEnd,YEnd,PathLength,ATPC,Frequency_y,_merge
2,AKMDAN_DERMA,AKMDAN BAR_RTN950A-25-MODU-1(RTNRF-1)-1,DERMA BAR_RTN950A-23-MODU-1(RTNRF-1)-1,8103.495,V,-1.95295,7.39630,-1.93781,7.21151,20.505,0.0,8103.495,both
4,AKMDAN_DERMA,AKMDAN BAR_RTN950A-25-MODU-2(RTNRF-2)-1,DERMA BAR_RTN950A-23-MODU-2(RTNRF-2)-1,8103.495,H,-1.95295,7.39630,-1.93781,7.21151,20.505,0.0,8103.495,both
7,AKMDAN_DERMA,AKMDAN_RTN950A_(KMO-TML)-25-MODU-1(RTNRF-1)-1,DERMA_RTN950A_(KMO-TML)-24-MODU-1(RTNRF-1)-1,6241.715,V,-1.95295,7.39630,-1.93781,7.21151,20.505,0.0,6241.715,both
9,AKMDAN_DERMA,AKMDAN_RTN950A_(KMO-TML)-25-MODU-2(RTNRF-2)-1,DERMA_RTN950A_(KMO-TML)-24-MODU-2(RTNRF-2)-1,6241.715,H,-1.95295,7.39630,-1.93781,7.21151,20.505,0.0,6241.715,both
12,ANTO_SHAMA,ANTO_RTN910A-27-MODU-1(RTNRF-1)-1,SHAMA_RTN910A-27-MODU-1(RTNRF-1)-1,5989.675,V,-1.65722,5.07578,-1.63895,5.00906,7.651,0.0,5989.675,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,AKRPON_MAMFE,AKRPON_RTN910A-27-MODU-2(RTNRF-2)-1,MAMFE_RTN910A-27-MODU-2(RTNRF-2)-1,18930.000,-,-0.08828,5.97910,-0.11979,5.94222,5.367,0.0,18930.000,both
1862,LOCOST_BAWKU,LOCOST_RTN910A-27-MODU-1(RTNRF-1)-1,BAWKU_RTN910A-24-MODU-1(RTNRF-1)-1,17920.000,V,-0.25444,11.05862,-0.22958,11.06200,2.742,0.0,17920.000,both
1863,LOCOST_BAWKU,LOCOST_RTN910A-27-MODU-2(RTNRF-2)-1,BAWKU_RTN910A-24-MODU-2(RTNRF-2)-1,17920.000,H,-0.25444,11.05862,-0.22958,11.06200,2.742,0.0,17920.000,both
1864,ADEISO_AMASAM,ADEISO_RTN910A-27-MODU-1(RTNRF-1)-1,AMASAM_RTN910A-2-27-MODU-1(RTNRF-1)-1,8222.095,V,-0.48194,5.79889,-0.29860,5.70973,22.574,0.0,8222.095,both


,Temp_ID,Monitored_ID,Far_end_ID,Frequency_x,Polarization,XStart,YStart,XEnd,YEnd,PathLength,ATPC,Frequency_y,_merge
0,AYETEP_SEGE,AYETEP_RTN950A-22-MODU-1(RTNRF-1)-1,SEGE_RTN950A-21-MODU-1(RTNRF-1)-1,8103.495,V,0.272880,5.779920,0.362760,5.874720,14.456,0.0,7792.175,both
1,AYETEP_SEGE,AYETEP_RTN950A-22-MODU-2(RTNRF-2)-1,SEGE_RTN950A-21-MODU-2(RTNRF-2)-1,8103.495,H,0.272880,5.779920,0.362760,5.874720,14.456,0.0,7792.175,both
3,AKMDAN_DERMA,AKMDAN BAR_RTN950A-25-MODU-1(RTNRF-1)-1,DERMA BAR_RTN950A-23-MODU-1(RTNRF-1)-1,8103.495,V,-1.952950,7.396300,-1.937810,7.211510,20.505,0.0,6241.715,both
5,AKMDAN_DERMA,AKMDAN BAR_RTN950A-25-MODU-2(RTNRF-2)-1,DERMA BAR_RTN950A-23-MODU-2(RTNRF-2)-1,8103.495,H,-1.952950,7.396300,-1.937810,7.211510,20.505,0.0,6241.715,both
6,AKMDAN_DERMA,AKMDAN_RTN950A_(KMO-TML)-25-MODU-1(RTNRF-1)-1,DERMA_RTN950A_(KMO-TML)-24-MODU-1(RTNRF-1)-1,6241.715,V,-1.952950,7.396300,-1.937810,7.211510,20.505,0.0,8103.495,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,AYIEBI_OFOASE,NaN,NaN,NaN,NaN,-1.122910,6.088170,-1.136780,6.168330,8.997,0.0,8222.095,right_only
1916,GRNHIL_SNAIRT,NaN,NaN,NaN,NaN,-2.302783,7.316661,-2.321769,7.326311,2.352,0.0,18930.000,right_only
1917,GARU_TEMPAN,NaN,NaN,NaN,NaN,-0.177670,10.850970,-0.129880,10.859130,5.303,0.0,17920.000,right_only
1918,MPRAES_WROBN,NaN,NaN,NaN,NaN,-0.761380,6.577220,-0.493310,6.484860,31.359,0.0,7792.175,right_only


In [6]:
# Fix inconsistency 2:
# Check if there are CMLs where the frequencies between Site 1 and Site 2 are swapped
swapped_frequencies = frequency_mismatch[frequency_mismatch['_merge'] == 'both'].copy()
id_frequency_pairs = set(zip(swapped_frequencies['Temp_ID'], swapped_frequencies['Frequency_x'], swapped_frequencies['Frequency_y']))

for idx, row in swapped_frequencies.iterrows():
    swapped_pairs = (row['Temp_ID'], row['Frequency_y'], row['Frequency_x'])  # swapped frequency_y/x
    if swapped_pairs in id_frequency_pairs and row['Frequency_x'] != row['Frequency_y']:
        swapped_frequencies.at[idx, 'Frequency_y'] = row['Frequency_x']

swapped_frequencies = swapped_frequencies[swapped_frequencies['Frequency_x'] == swapped_frequencies['Frequency_y']]

# update the matched and mismatched frequencies
frequency_match = pd.concat([frequency_match, swapped_frequencies], ignore_index=True)
frequency_mismatch = frequency_mismatch.drop(swapped_frequencies.index)

In [7]:
# Fix inconsistency 3:
# Allow for 500 MHz difference in frequencies
frequency_buffer = frequency_mismatch[(frequency_mismatch['_merge'] == 'both') & (abs(frequency_mismatch['Frequency_x'] - frequency_mismatch['Frequency_y']) <= 500)]

# update the matched and mismatched frequencies
frequency_match = pd.concat([frequency_match, frequency_buffer], ignore_index=True)
frequency_mismatch = frequency_mismatch.drop(frequency_buffer.index)

In [8]:
# Drop column Frequncy_y and change name of Frequency_x to Frequency
frequency_match = frequency_match.drop(columns=['Frequency_y', '_merge'])
frequency_match = frequency_match.rename(columns={'Frequency_x': 'Frequency'})

In [9]:
# Summary
print('The total number of IDs in both metadata files together is:', len(all_metadata))
print('Based on matching IDs in the two metadata files, the number of available sub-links is:', len(all_metadata[all_metadata['_merge'] == 'both']))
print('Based on a more strict matching of IDs and frequencies, the number of available sub-links is:', len(frequency_match))

print('Number of sub-links with matching IDs but mismatching frequencies (excl. missing frequency data):', len(frequency_mismatch[frequency_mismatch['_merge'] == 'both'].dropna(subset=['Frequency_x', 'Frequency_y'])))

The total number of IDs in both metadata files together is: 1920
Based on matching IDs in the two metadata files, the number of available sub-links is: 1780
Based on a more strict matching of IDs and frequencies, the number of available sub-links is: 1675
Number of sub-links with matching IDs but mismatching frequencies (excl. missing frequency data): 40


In [10]:
matched_metadata = frequency_match.copy()

In [11]:
matched_metadata

,Temp_ID,Monitored_ID,Far_end_ID,Frequency,Polarization,XStart,YStart,XEnd,YEnd,PathLength,ATPC
0,AKMDAN_DERMA,AKMDAN BAR_RTN950A-25-MODU-1(RTNRF-1)-1,DERMA BAR_RTN950A-23-MODU-1(RTNRF-1)-1,8103.495,V,-1.952950,7.39630,-1.937810,7.211510,20.505,0.0
1,AKMDAN_DERMA,AKMDAN BAR_RTN950A-25-MODU-2(RTNRF-2)-1,DERMA BAR_RTN950A-23-MODU-2(RTNRF-2)-1,8103.495,H,-1.952950,7.39630,-1.937810,7.211510,20.505,0.0
2,AKMDAN_DERMA,AKMDAN_RTN950A_(KMO-TML)-25-MODU-1(RTNRF-1)-1,DERMA_RTN950A_(KMO-TML)-24-MODU-1(RTNRF-1)-1,6241.715,V,-1.952950,7.39630,-1.937810,7.211510,20.505,0.0
3,AKMDAN_DERMA,AKMDAN_RTN950A_(KMO-TML)-25-MODU-2(RTNRF-2)-1,DERMA_RTN950A_(KMO-TML)-24-MODU-2(RTNRF-2)-1,6241.715,H,-1.952950,7.39630,-1.937810,7.211510,20.505,0.0
4,ANTO_SHAMA,ANTO_RTN910A-27-MODU-1(RTNRF-1)-1,SHAMA_RTN910A-27-MODU-1(RTNRF-1)-1,5989.675,V,-1.657220,5.07578,-1.638950,5.009060,7.651,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1670,LAGBES_WAWALE,LAGBES_RTN950A-24-MODU-2(RTNRF-2)-1,WAWALE_RTN950A_2-24-MODU-2(RTNRF-2)-1,8222.094,H,-0.563780,10.40354,-0.799333,10.337356,26.815,0.0
1671,LAGBES_WAWALE,LAGBES_RTN950A-26-MODU-1(RTNRF-1)-1,WAWALE_RTN950A_2-26-MODU-1(RTNRF-1)-1,8222.094,V,-0.563780,10.40354,-0.799333,10.337356,26.815,0.0
1672,LAGBES_WAWALE,LAGBES_RTN950A-26-MODU-2(RTNRF-2)-1,WAWALE_RTN950A_2-26-MODU-2(RTNRF-2)-1,8222.094,H,-0.563780,10.40354,-0.799333,10.337356,26.815,0.0
1673,KRACHI_NKOTRE,KRACHI_RTN950A-23-MODU-1(RTNRF-1)-1,NKOTRE_RTN950A-23-MODU-1(RTNRF-1)-1,5989.674,-,-0.050108,7.79388,-0.338610,7.793060,31.821,0.0


### Cleaning the metadata

**ATPC** <br>
Some rows contain Automatic Transmit Power Control (ATPC), but most don't. Since it is unknown how and when the ATPC is applied, and RAINLINK is currently not suited to handle changes in signal levels due to ATPC, we discard these sub-links. 

In [12]:
print('Number of sub-links with ATPC:', len(matched_metadata[matched_metadata['ATPC'] != 0].drop_duplicates(subset=['Monitored_ID', 'Far_end_ID'])))
matched_metadata = matched_metadata[matched_metadata['ATPC'] == 0]

Number of sub-links with ATPC: 59


In [13]:
# remove non-numeric values from numeric columns
num_cols = ['Frequency', 'XStart', 'YStart', 'XEnd', 'YEnd', 'PathLength']
matched_metadata.loc[:, num_cols] = matched_metadata[num_cols].apply(pd.to_numeric, errors='coerce')
drop_rows = matched_metadata[num_cols].isna().any(axis=1)
print(f"Drop {matched_metadata[drop_rows].index.to_list()} rows because these contain non-numeric values.")
matched_metadata = matched_metadata[~drop_rows]

# drop rows that have NaNs in required columns
matched_metadata = matched_metadata.dropna(subset=['Frequency', 'Polarization', 'XStart', 'YStart', 'XEnd', 'YEnd'])
print('Number of sub-links for which we have all the required signal level data and metadata:', len(matched_metadata.drop_duplicates(subset=['Monitored_ID', 'Far_end_ID'])))


# remove zero frequencies or path lengths
matched_metadata = matched_metadata[matched_metadata['Frequency'] > 0]
matched_metadata = matched_metadata[matched_metadata['PathLength'] > 0]

Drop [] rows because these contain non-numeric values.
Number of sub-links for which we have all the required signal level data and metadata: 1548


In [16]:
# optionally, save to file
matched_metadata.to_csv('../data/ftp/matched_metadata.csv', index=False)

## Coupling metadata with signal level data

The sections above, combining the metadata files, are static, as long as the metadata is not updated. In case of continuous data sharing, the part below will have to be rerun everytime new data comes in to couple the static metadata to the new signal level data. 

In [2]:
matched_metadata = pd.read_csv('../data/ftp/matched_metadata.csv')

### Loading signal level data from ftp server

Signal level data from the ftp server comes in approximately every 3 hours. The filenames are in the format: <br>
*Schedule_pfm_SDH_20250829064902281472061927872_1.txt* <br>
*Schedule_pfm_SDH_20250829034454281472860733888_1.txt* <br>
*Schedule_pfm_SDH_20250829004621281472135393728_1.txt*, <br>
where *202508290046* represents the YYYYMMDDHHMM. <br>

Using the default RAINLINK parameters, we need to be able to 'look back' at least 24 hours adequately perform the wet-dry classification (RAINLINK step 2) and the reference level determination (RAINLINK step 3). For reference see: https://github.com/overeem11/RAINLINK/blob/master/RAINLINK1.31.pdf. 

For more reliability, and to be able to compute the path-averaged rainfall intensity for any given 15-minute interval in a 24-hour period, we load the previous 51 hours (48 + 3 hrs) from the ftp server. The reason to use 51 is that in the files that are uploaded to the ftp just after 00:00 hours, there is data from both the previous and the current day. 

In [3]:
local_ftp_dir = '../../ATGhana_ftp/ATGhana'

In [4]:
# With help from ChatGPT: 

# Optional: set a manual datetime from which to count back, or None to use newest file
manual_latest_dt = datetime(2025, 8, 13, 0, 0)  # Example: datetime(2025, 8, 29, 12, 0)

# Function to extract datetime from filename
def extract_datetime_from_filename(fname):
    # Example: Schedule_pfm_SDH_20250812004105281472818770368_1
    parts = fname.split("_")
    if len(parts) < 4:
        return None  # unexpected filename
    
    timestamp = parts[3]  # "20250812004105281472818770368"
    try:
        # Take first 10 chars: YYYYMMDDHH
        dt = datetime.strptime(timestamp[:10], "%Y%m%d%H")
        return dt
    except ValueError:
        return None

# Step 1: Get file timestamps
file_names = os.listdir(local_ftp_dir)
file_datetimes = [(extract_datetime_from_filename(f), f) for f in file_names]
file_datetimes = [(dt, f) for dt, f in file_datetimes if dt is not None]

# Step 2: Determine “latest” datetime
if manual_latest_dt is not None:
    latest_dt = manual_latest_dt
else:
    latest_dt, latest_file = max(file_datetimes, key=lambda x: x[0])

# Step 3: Define cutoff time
cutoff_time = latest_dt - timedelta(hours=51)

# Step 4: Filter files between cutoff_time and latest_dt and sort chronologically
recent_files = [(dt, f) for dt, f in file_datetimes if cutoff_time <= dt <= latest_dt]
recent_files.sort(key=lambda x: x[0])

# Step 5: Read and combine into one dataframe
df_list = []
for _, fname in recent_files:
    fpath = os.path.join(local_ftp_dir, fname)
    try:
        # df = pd.read_csv(fpath)  
        df = pd.read_csv(fpath, header=0, sep='\t') # assumes headers are present and identical
        df["source_file"] = fname   # optional: track origin file
        df_list.append(df)
    except Exception as e:
        print(f"⚠️ Could not read {fname}: {e}")

if df_list:
    ftp_data = pd.concat(df_list, ignore_index=True)
    print(f"✅ Combined dataframe shape: {ftp_data.shape} "
          f"(from {len(df_list)} files)")
else:
    ftp_data = pd.DataFrame()
    print("⚠️ No valid files found to combine.")



✅ Combined dataframe shape: (3682352, 29) (from 18 files)


For each unique combination of timestamp and ID, the signal level data contains 8 rows (```TSL_MIN```, ```TSL_MAX```, ```TSL_AVG```, ```TSL_CUR```, ```RSL_MIN```, ```RSL_MAX```, ```RSL_AVG``` and ```RSL_CUR```). 
Since the signal level data is generated from the point of view of a node, the TSL is the transmitted signal level from the ```Monitored_ID``` to the ```Far_end_ID```, and the RSL is the signal level received at the ```Monitored_ID```, coming from the ```Far_end_ID```. Hence, the TSL and RSL values belong to different sub-links!

Currently the signal level data files from the FTP server do not contain information on the Far End. To be able to couple the RSL data to the metadata from the correct sub-link (coming from the other direction) we need to retrieve the ```Far_end_ID``` from the metadata, based on the ```Monitored_ID```, which is unique. 

We therefore create two different dataframes, one for TSL and one for RSL and merge those later. For the RSL dataframe we swap the ```Monitored_ID``` and ```Far_end_ID```. 




In [5]:
# Construct an ID in such a format to be able to distinguish between sublinks, antennas and polarizations across a single path
ftp_data['Monitored_ID'] = (ftp_data['NEName'].astype(str) + '-' + 
                            ftp_data['BrdID'].astype(str) + '-' + 
                            ftp_data['BrdName'].astype(str) + '-' + 
                            ftp_data['PortNO'].astype(str) + '(' + 
                            ftp_data['PortName'].astype(str) + ')-' + 
                            ftp_data['PathID'].astype(str)
)

We add a polarization column to the signal level data to use as a double check when coupling the metadata. We know from the file *Microwave_Link_Report_* that ```MODU-1(RTNRF-1)``` refers to vertically polarized signals and ```MODU-2(RTNRF-2)``` to horizontally polarized signals. 

In [6]:
def extract_polarization(x):
    if "MODU-" in x:
        modu_part = x.split("MODU-")[1]  
        modu_number = modu_part[0]      
        return {'1': 'V', '2': 'H'}.get(modu_number, None)
    return None

ftp_data['Polarization'] = ftp_data['Monitored_ID'].apply(extract_polarization)

### The coupling

In [7]:
cml_data = pd.merge(ftp_data, matched_metadata, on=['Monitored_ID'])

Not all polarizations between the metadata and the CML data match because in some cases the metadata misses polarization data because it was not in the metadata file *Microwave_Link_Report_*. However, since we can deduce the polarization from the ```Monitored_ID``` with a fair amount of certainty, to minimize the number of links discarded, when the polarization is missing from the metadata we use the polarization deduced in the CML data file. 


In [8]:
print('Percentage of rows with unmatched polarizations:', len(cml_data[cml_data['Polarization_x'] != cml_data['Polarization_y']]) / len(cml_data) * 100)
cml_data['Polarization'] = np.where((cml_data['Polarization_x'] != cml_data['Polarization_y']) & ~(cml_data['Polarization_x'].isin(['H', 'V'])), 
                                      cml_data['Polarization_y'],
                                      cml_data['Polarization_x'])

Percentage of rows with unmatched polarizations: 1.591943873079841


In [9]:
cml_data.columns

Index(['ONEID', 'ONEName', 'NEID', 'NEName', 'NEType', 'ShelfID', 'BrdID',
       'BrdType', 'BrdName', 'PortID', 'PortNO', 'PortName', 'MOType',
       'FBName', 'PathID', 'EventID', 'EventName', 'Period', 'EndTime',
       'Value', 'UnitName', 'PMParameterName', 'PMLocationID', 'PMLocation',
       'UpLevel', 'DownLevel', 'ResultOfLevel', 'Unnamed: 27', 'source_file',
       'Monitored_ID', 'Polarization_x', 'Temp_ID', 'Far_end_ID', 'Frequency',
       'Polarization_y', 'XStart', 'YStart', 'XEnd', 'YEnd', 'PathLength',
       'ATPC', 'Polarization'],
      dtype='object')

In [10]:
cml_data = cml_data.drop(columns=['ONEID', 'ONEName', 'NEID', 'NEType', 
                                  'NEName', 'BrdID', 'BrdName', 'PortNO', 'PortName', 'PathID',
                                  'ShelfID', 'BrdType','PortID', 'MOType', 
                                  'FBName', 'EventID', 'PMParameterName', 'PMLocationID', 
                                  'PMLocation', 'UpLevel', 'DownLevel', 'ResultOfLevel', 
                                  'Unnamed: 27', 'Polarization_x', 'Temp_ID', 'Polarization_y', 
                                  'ATPC'])

In [12]:
# Columns to group by
group_columns = [
    'Monitored_ID', 'Far_end_ID', 'Polarization', 'Period', 'EndTime'
]

#########################################
def get_event_value_or_return_nan(group, event_name):
    """Get the value from the correct column based on the event name."""
    row = group[group['EventName'] == event_name]
    return row['Value'].values[0] if not row.empty else np.nan
#########################################

# TSL
df_tsl = cml_data.copy()
flattened_tsl_rows = []

for group_keys, group_data in df_tsl.groupby(group_columns):
    row = dict(zip(group_columns, group_keys))

    for event in ['TSL_MIN', 'TSL_MAX', 'TSL_CUR', 'TSL_AVG']:
        row[event] = get_event_value_or_return_nan(group_data, event)

    flattened_tsl_rows.append(row)

tsl_flattened = pd.DataFrame(flattened_tsl_rows)

# RSL
# 'swap' the Monitored and Far End IDs first
df_rsl = cml_data.copy()
df_rsl = df_rsl.rename(columns={
    'Monitored_ID': 'Far_end_ID',
    'Far_end_ID': 'Monitored_ID',
})

flattened_rsl_rows = []

for group_keys, group_data in df_rsl.groupby(group_columns):
    row = dict(zip(group_columns, group_keys))

    for event in ['RSL_MIN', 'RSL_MAX', 'RSL_CUR', 'RSL_AVG']:
        row[event] = get_event_value_or_return_nan(group_data, event)

    flattened_rsl_rows.append(row)

rsl_flattened = pd.DataFrame(flattened_rsl_rows)


# Merge the TSL and RSL dataframes
cml_data_flattened = pd.merge(tsl_flattened, rsl_flattened, on=group_columns)

# Add the other metadata columns back to the flattened dataframe
metadata_cols = ['Frequency', 'XStart', 'YStart', 'XEnd', 'YEnd', 'PathLength']
cml_data_unique_metadata = cml_data[group_columns + metadata_cols].drop_duplicates()
cml_data_flattened = pd.merge(cml_data_flattened, cml_data_unique_metadata, how='left', on=group_columns)



In [13]:
# Summary
print('Number of sub-links for which we have (part of) the signal level data:', len(ftp_data.drop_duplicates(subset=['Monitored_ID'])))
print('Number of sub-links for which we have (part of) the signal level data and metadata:', len(cml_data.drop_duplicates(subset=['Monitored_ID', 'Far_end_ID'])))
print('Number of sub-links for which we have missing TSL data for some timestamps:', len(cml_data_flattened[cml_data_flattened[['TSL_MAX', 'TSL_CUR', 'TSL_MIN', 'TSL_AVG']].isna().all(axis=1)].drop_duplicates(subset=['Monitored_ID', 'Far_end_ID'])))
print('Number of sub-links for which we have missing RSL data for some timestamps:', len(cml_data_flattened[cml_data_flattened[['RSL_MAX', 'RSL_CUR', 'RSL_MIN', 'RSL_AVG']].isna().all(axis=1)].drop_duplicates(subset=['Monitored_ID', 'Far_end_ID'])))
print('Number of sub-links for which we have all signal level data for some timestamps:', len(cml_data_flattened.dropna().drop_duplicates(subset=['Monitored_ID', 'Far_end_ID'])))
print('Number of sub-links for which we have no TSL data for some timestamps:', cml_data_flattened.groupby(['Monitored_ID', 'Far_end_ID'])[['TSL_MIN', 'TSL_MAX', 'TSL_AVG', 'TSL_CUR']].apply(lambda g: g.isna().all().all()).sum())
print('Number of sub-links for which we have no RSL data for some timestamps:', cml_data_flattened.groupby(['Monitored_ID', 'Far_end_ID'])[['RSL_MIN', 'RSL_MAX', 'RSL_AVG', 'RSL_CUR']].apply(lambda g: g.isna().all().all()).sum())


Number of sub-links for which we have (part of) the signal level data: 1639
Number of sub-links for which we have (part of) the signal level data and metadata: 1429
Number of sub-links for which we have missing TSL data for some timestamps: 0
Number of sub-links for which we have missing RSL data for some timestamps: 0
Number of sub-links for which we have all signal level data for some timestamps: 1320
Number of sub-links for which we have no TSL data for some timestamps: 0
Number of sub-links for which we have no RSL data for some timestamps: 0


In [26]:
linkdata = cml_data_flattened.copy()

### TSL filter 
Fluctuations in TSL are generally small compared to fluctuations in RSL. Nevertheless, since a fluctuation in TSL does influence the specific attenuation along the path, if it is not constant, it has to be known, and if it is not known, the specific attenuation along the path cannot be determined, and the sub-link can thus not be used. <br>
Note: if fluctuations in TSL are generally small, and removing sub-links without TSL data leads to high data loss, it is also possible to not discard these. Alternatively also a TSL filter could be applied that only removes timestamps if the TSL values fluctuate too much (<0.25 dB used in previous work from Sri Lanka).

Additionally, a TSL(min, max, avg and cur) of -55 dBm often points to a link in "stand-by" mode. What this means exactly is unclear (perhaps that there is no bandwidth across the signal?). Occasionally the value of -55 dBm only appears for the TSL_min, and not all the other variables (max/avg/cur). This could possibly point to the device having gone in some form of "stand-by" mode sometime in the previous 15 minutes. Despite this "stand-by" mode there is often still a signal being transmitted as most of these links do record an RSL at the other end. Hence, for now the choice is made to keep these links with a constant TSL of -55 dBm. 

In [27]:
# To be safe we also best drop rows with no TSL data
linkdata = linkdata.dropna(subset=['TSL_MIN', 'TSL_MAX', 'TSL_AVG', 'TSL_CUR'])
print('Number of sub-links for which we have all the required signal level data and metadata (but not necessarily for all timestamps):', len(linkdata.drop_duplicates(subset=['Monitored_ID', 'Far_end_ID'])))

print('Percentage of rows for which all TSL variables are -55:', len(linkdata[linkdata[['TSL_MIN', 'TSL_MAX', 'TSL_AVG', 'TSL_CUR']].isin([-55]).all(axis=1)]) / len(linkdata) * 100)
print('Percentage of rows for which only TSL_MIN is -55:', len(linkdata[(linkdata['TSL_MIN'] == -55) & (linkdata['TSL_MAX'] != -55) & (linkdata['TSL_AVG'] != -55) & (linkdata['TSL_CUR'] != -55)]) / len(linkdata) * 100)


Number of sub-links for which we have all the required signal level data and metadata (but not necessarily for all timestamps): 1320
Percentage of rows for which all TSL variables are -55: 19.43998221482767
Percentage of rows for which only TSL_MIN is -55: 0.17964458341102393


### Turn the data into RAINLINK format

Since TSL is not constant, we will need to subtract TSL from RSL before running RAINLINK to get the actual values of attenuation along the path! Because the minimum and maximum TSL do not necessarily coincide with the minimum and maximum RSL in the previous 15 minutes, we use the average TSL to subtract from the RSL, as this is the most representative for the 15 minutes. 

In [28]:
linkdata['Pmin'] = linkdata['RSL_MIN'] - linkdata['TSL_AVG']
linkdata['Pmax'] = linkdata['RSL_MAX'] - linkdata['TSL_AVG']

RAINLINK only takes one ID variable. To ensure that each ID is unique we join the ```Monitored_ID``` with the ```Far_end_ID``` using '>>' to create a unique ID for each and every connection between two sites. 

In [29]:
linkdata['ID'] = linkdata['Monitored_ID'] + '>>' + linkdata['Far_end_ID']

**Note**: the individual parts that make up the ID are not changed so that these can be traced back in the metadata Excel files if needed. This means that currently there can be special characters or blank spaces present in the IDs, for example due to typos in the metadata Excel files. <br>
If blank spaces are undesired these can always be replaced with '_'. 

In [ ]:
# Show ID with any character that is not "a letter, digit, -, _, ( or )"
special_character_ids = matched_metadata.loc[matched_metadata["Monitored_ID"].str.contains(r"[^A-Za-z0-9_()-]", na=False), "Monitored_ID"].unique()
display(special_character_ids)

array(['AKMDAN BAR_RTN950A-25-MODU-1(RTNRF-1)-1',
       'AKMDAN BAR_RTN950A-25-MODU-2(RTNRF-2)-1',
       'AKMDAN BAR_RTN950A-22-MODU-1(RTNRF-1)-1',
       'AKMDAN BAR_RTN950A-22-MODU-2(RTNRF-2)-1',
       'ABUFAW BAR_RTN950A-23-MODU-1(RTNRF-1)-1',
       'ABUFAW BAR_RTN950A-23-MODU-2(RTNRF-2)-1',
       'TEMA NOC_RTN950A_2-21-MODU-1(RTNRF-1)-1',
       'TEMA NOC_RTN950A_2-21-MODU-2(RTNRF-2)-1',
       'ABURI.2A_RTN 950A-23-MODU-1(RTNRF-1)-1',
       'ABURI.2A_RTN 950A-23-MODU-2(RTNRF-2)-1',
       'TAGBC_RTN-2_UE&SALAGA_BB-24-MODU-1(RTNRF-1)-1',
       'TAGBC_RTN-2_UE&SALAGA_BB-24-MODU-2(RTNRF-2)-1',
       'TAGBC_RTN-2_UE&SALAGA_BB-26-MODU-1(RTNRF-1)-1',
       'TAGBC_RTN-2_UE&SALAGA_BB-26-MODU-2(RTNRF-2)-1',
       'AKMDAN BAR_RTN950A-24-MODU-1(RTNRF-1)-1',
       'AKMDAN BAR_RTN950A-24-MODU-2(RTNRF-2)-1',
       'TAGBC_RTN-2_UE&SALAGA_BB-23-MODU-1(RTNRF-1)-1',
       'TAGBC_RTN-2_UE&SALAGA_BB-23-MODU-2(RTNRF-2)-1',
       'TAGBC_RTN-2_UE&SALAGA_BB-25-MODU-1(RTNRF-1)-1',
       'TA

In [31]:
# Drop rows that are not needed for RAINLINK
linkdata = linkdata.drop(columns=['Monitored_ID',
                                  'Far_end_ID',
                                  'Period'])

# change names of the remaining columns to RAINLINK format
linkdata = linkdata.rename(columns={
    'EndTime': 'DateTime'
})

# put the frequency and datetime in RAINLINK format
linkdata['Frequency'] = linkdata['Frequency'] / 1000 # convert to GHz
linkdata['DateTime'] = pd.to_datetime(linkdata['DateTime']).dt.strftime('%Y%m%d%H%M')

# and re-order columns and keep only PathLength and TSL_AVG as extra columns
order_columns = ['Frequency', 'DateTime', 'Pmin', 'Pmax', 'XStart', 'YStart', 'XEnd', 'YEnd', 'ID',  
                 'Polarization', 'PathLength', 'TSL_AVG']
linkdata = linkdata[order_columns]

In [32]:
# save to file
linkdata.to_csv(f'../data/ftp/Linkdata_AT_{latest_dt.strftime("%Y%m%d")}.dat', sep=',', index=False)